In [4]:
import torch
from torchvision import transforms
transform = transforms.functional.resize

x = torch.ones(32, 2, 100, 110)
transforms.Normalize(mean=[1, 2], std=[1, 2])(x).shape

torch.Size([32, 2, 100, 110])

In [5]:
from config import get_data
import yaml
config_file = './configs/structured.yaml'
with open(config_file, 'r') as f:
    cfg = yaml.load(f, Loader=yaml.Loader)

data = get_data(cfg['data'])

In [17]:
x = torch.ones(10, 20, 30)
torch.stack((x, x)).shape

torch.Size([2, 10, 20, 30])

In [7]:
import matplotlib.pyplot as plt
from datasets.structured import StructuredModule
data_cfg = {
    'data_dir': './data/npy',
    'batch_size': 64,
}
data_module = StructuredModule(**data_cfg)
data_module.setup()
train_loader = data_module.train_dataloader()

    

In [8]:
for batch in train_loader:
    [x, c], y = batch
    u, v, p = y.permute(1, 0, 2, 3)
    
    break

In [9]:
plt.hist(u.flatten())



ValueError: x must have 2 or fewer dimensions

In [10]:
plt.hist(v.flatten())

torch.Size([64, 118, 517])

In [ ]:
plt.hist(p.flatten())